In [ ]:
!pip install transformers

     |████████████████████████████████| 2.0MB 14.9MB/s 
     |████████████████████████████████| 890kB 52.1MB/s 
     |████████████████████████████████| 3.2MB 51.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=bc93d9d2d5820291262a3fbbd1c0ae4242bc8e65aca13da2f46346348a0c45b9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
# Importing stock ml libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [ ]:
import pandas as pd
import numpy as np
import torch
import torchvision
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import nltk
from nltk.tokenize import word_tokenize
import torch
from collections import defaultdict
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:

df_te = pd.read_csv('/content/drive/My Drive/ACDTESTReg.csv')
df_te = df_te.rename(columns={'sentence': 'text', 'aspect': 'aspect_term', 'sentiment': 'Class'})
df_te.head(50)



,text,aspect_term,Class
0,the bread is top notch as well.,food,positive
1,i have to say they have one of the fastest del...,service,positive
2,food is always fresh and hot- ready to eat!,food,positive
3,did i mention that the coffee is outstanding?,food,positive
4,"certainly not the best sushi in new york, howe...",food,conflict
5,"certainly not the best sushi in new york, howe...",ambience,positive
6,"i trust the people at go sushi, it never disap...",misc,positive
7,"straight-forward, no surprises, very decent ja...",food,positive
8,"best spicy tuna roll, great asian salad.",food,positive
9,try the rose roll (not on menu).,food,positive


In [ ]:
#df_te.Class = df_te.Class.replace('conflict', 'neutral') 
df_te.drop( df_te[ df_te['Class'] == 'conflict' ].index , inplace=True)
df_te = df_te.reset_index(drop=True)
df_te.aspect_term = df_te.aspect_term.replace('misc', 'miscellaneous') 
df_test = df_te
df_test.head(50)

,text,aspect_term,Class
0,the bread is top notch as well.,food,positive
1,i have to say they have one of the fastest del...,service,positive
2,food is always fresh and hot- ready to eat!,food,positive
3,did i mention that the coffee is outstanding?,food,positive
4,"certainly not the best sushi in new york, howe...",ambience,positive
5,"i trust the people at go sushi, it never disap...",miscellaneous,positive
6,"straight-forward, no surprises, very decent ja...",food,positive
7,"best spicy tuna roll, great asian salad.",food,positive
8,try the rose roll (not on menu).,food,positive
9,"i love the drinks, esp lychee martini, and the...",food,positive


In [ ]:
from sklearn import preprocessing

# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'Country'. 
df_test['Class']= label_encoder.fit_transform(df_test['Class']) 
df_test.head()

onehotencoder = preprocessing.OneHotEncoder()
#reshape the 1-D country array to 2-D as fit_transform expects 2-D and finally fit the object 
X = onehotencoder.fit_transform(df_test.Class.values.reshape(-1,1)).toarray()
#To add this back into the original dataframe 
dfOneHot = pd.DataFrame(X, columns = ["Class_"+str(int(i)) for i in range(df_test.shape[1])]) 
df_test = pd.concat([df_test, dfOneHot], axis=1)
#droping the country column 
df_test= df_test.drop(['Class'], axis=1) 
#printing to verify 
df_test.head()

,text,aspect_term,Class_0,Class_1,Class_2
0,the bread is top notch as well.,food,0.0,0.0,1.0
1,i have to say they have one of the fastest del...,service,0.0,0.0,1.0
2,food is always fresh and hot- ready to eat!,food,0.0,0.0,1.0
3,did i mention that the coffee is outstanding?,food,0.0,0.0,1.0
4,"certainly not the best sushi in new york, howe...",ambience,0.0,0.0,1.0


In [ ]:
df_test['list'] = df_test[df_test.columns[2:]].values.tolist()
new_df_test = df_test[['text', 'aspect_term', 'list']].copy()
new_df_test.head(20)

,text,aspect_term,list
0,the bread is top notch as well.,food,"[0.0, 0.0, 1.0]"
1,i have to say they have one of the fastest del...,service,"[0.0, 0.0, 1.0]"
2,food is always fresh and hot- ready to eat!,food,"[0.0, 0.0, 1.0]"
3,did i mention that the coffee is outstanding?,food,"[0.0, 0.0, 1.0]"
4,"certainly not the best sushi in new york, howe...",ambience,"[0.0, 0.0, 1.0]"
5,"i trust the people at go sushi, it never disap...",miscellaneous,"[0.0, 0.0, 1.0]"
6,"straight-forward, no surprises, very decent ja...",food,"[0.0, 0.0, 1.0]"
7,"best spicy tuna roll, great asian salad.",food,"[0.0, 0.0, 1.0]"
8,try the rose roll (not on menu).,food,"[0.0, 0.0, 1.0]"
9,"i love the drinks, esp lychee martini, and the...",food,"[0.0, 0.0, 1.0]"


In [ ]:

df = pd.read_csv('/content/drive/My Drive/ACDReg.csv')
df = df.rename(columns={'sentence': 'text', 'aspect': 'aspect_term', 'sentiment': 'Class'})
df.head(50)



,text,aspect_term,Class
0,but the waitstaff was so horrible to us.,service,negative
1,"to be completely fair, the only redeeming fact...",food,positive
2,"to be completely fair, the only redeeming fact...",misc,negative
3,"the food is uniformly exceptional, with a very...",food,positive
4,where gabriela personally greets you and recom...,service,positive
5,"for those that go once and don't enjoy it, all...",misc,positive
6,"not only was the food outstanding, but the lit...",food,positive
7,"not only was the food outstanding, but the lit...",service,positive
8,it is very overpriced and not very tasty.,food,negative
9,it is very overpriced and not very tasty.,price,negative


In [ ]:
df.shape

(3713, 3)

In [ ]:
#df.Class = df.Class.replace('conflict', 'neutral') 
df.drop( df[ df['Class'] == 'conflict' ].index , inplace=True)
df = df.reset_index(drop=True)
df.aspect_term = df.aspect_term.replace('misc', 'miscellaneous') 
df.shape
df.head(50)

,text,aspect_term,Class
0,but the waitstaff was so horrible to us.,service,negative
1,"to be completely fair, the only redeeming fact...",food,positive
2,"to be completely fair, the only redeeming fact...",miscellaneous,negative
3,"the food is uniformly exceptional, with a very...",food,positive
4,where gabriela personally greets you and recom...,service,positive
5,"for those that go once and don't enjoy it, all...",miscellaneous,positive
6,"not only was the food outstanding, but the lit...",food,positive
7,"not only was the food outstanding, but the lit...",service,positive
8,it is very overpriced and not very tasty.,food,negative
9,it is very overpriced and not very tasty.,price,negative


In [ ]:
from sklearn import preprocessing

# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'Country'. 
df['Class']= label_encoder.fit_transform(df['Class']) 
df.head()

onehotencoder = preprocessing.OneHotEncoder()
#reshape the 1-D country array to 2-D as fit_transform expects 2-D and finally fit the object 
X = onehotencoder.fit_transform(df.Class.values.reshape(-1,1)).toarray()
#To add this back into the original dataframe 
dfOneHot = pd.DataFrame(X, columns = ["Class_"+str(int(i)) for i in range(df.shape[1])]) 
df = pd.concat([df, dfOneHot], axis=1)
#droping the country column 
df= df.drop(['Class'], axis=1) 
#printing to verify 
df.head()

,text,aspect_term,Class_0,Class_1,Class_2
0,but the waitstaff was so horrible to us.,service,1.0,0.0,0.0
1,"to be completely fair, the only redeeming fact...",food,0.0,0.0,1.0
2,"to be completely fair, the only redeeming fact...",miscellaneous,1.0,0.0,0.0
3,"the food is uniformly exceptional, with a very...",food,0.0,0.0,1.0
4,where gabriela personally greets you and recom...,service,0.0,0.0,1.0


In [ ]:
df['list'] = df[df.columns[2:]].values.tolist()
new_df = df[['text', 'aspect_term', 'list']].copy()
new_df.head(20)

,text,aspect_term,list
0,but the waitstaff was so horrible to us.,service,"[1.0, 0.0, 0.0]"
1,"to be completely fair, the only redeeming fact...",food,"[0.0, 0.0, 1.0]"
2,"to be completely fair, the only redeeming fact...",miscellaneous,"[1.0, 0.0, 0.0]"
3,"the food is uniformly exceptional, with a very...",food,"[0.0, 0.0, 1.0]"
4,where gabriela personally greets you and recom...,service,"[0.0, 0.0, 1.0]"
5,"for those that go once and don't enjoy it, all...",miscellaneous,"[0.0, 0.0, 1.0]"
6,"not only was the food outstanding, but the lit...",food,"[0.0, 0.0, 1.0]"
7,"not only was the food outstanding, but the lit...",service,"[0.0, 0.0, 1.0]"
8,it is very overpriced and not very tasty.,food,"[1.0, 0.0, 0.0]"
9,it is very overpriced and not very tasty.,price,"[1.0, 0.0, 0.0]"


In [ ]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 100
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 8
EPOCHS = 4
#epochs = 3
LEARNING_RATE = 2e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',return_dict=False)

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.aspect_term = dataframe.aspect_term
        self.targets = self.data.list
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())
        aspect_term = str(self.aspect_term[index])
        aspect_term = " ".join(aspect_term.split())
        #print(text)
        inputs = self.tokenizer.encode_plus(
            text,
            aspect_term,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
# Creating the dataset and dataloader for the neural network

train_size = 1.00
test_size = 1.00
train_dataset=new_df.sample(frac=train_size,random_state=200)
test_dataset=new_df_test.sample(frac=test_size,random_state=200)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (3518, 6)
TRAIN Dataset: (3518, 3)
TEST Dataset: (973, 3)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0,
                
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 3)
    
    def forward(self, ids, mask, token_type_ids):
        _, pooled_output= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(pooled_output)
        output = self.l3(output_2)
        return output

fchidden = 256
hiddendim_lstm = 256
embeddim = 768
numlayers = 12
numclasses = 3
num_heads = 2

class Bert_LSTM(torch.nn.Module):
    def __init__(self, numclasses):
        super(Bert_LSTM, self).__init__()
        self.numclasses = numclasses
        self.embeddim = embeddim
        self.numlayers = numlayers
        self.hiddendim_lstm = hiddendim_lstm
        self.dropout = torch.nn.Dropout(0.3)

        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        print("BERT Model Loaded")
        self.lstm = torch.nn.LSTM(self.embeddim, self.hiddendim_lstm, bidirectional = False, batch_first=True) # noqa
        self.fc = torch.nn.Linear(self.hiddendim_lstm, self.numclasses)

    def forward(self, ids, mask, token_type_ids):
      last_hidden_state, pooled_output = self.bert(ids, attention_mask = mask, token_type_ids = token_type_ids)
      #print(last_hidden_state.shape)
      #print(pooled_output.shape)
      out, _ = self.lstm(last_hidden_state, None)
      #print(out.shape)
      out = self.dropout(out[:, -1, :])
      #print(out.shape)
      out = self.fc(out)
      return out


class Bert_GRU(torch.nn.Module):
    def __init__(self, numclasses):
        super(Bert_GRU, self).__init__()
        self.numclasses = numclasses
        self.embeddim = embeddim
        self.numlayers = numlayers
        self.hiddendim_lstm = hiddendim_lstm
        self.dropout = torch.nn.Dropout(0.3)

        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        print("BERT Model Loaded")
        self.Gru = torch.nn.GRU(self.embeddim, self.hiddendim_lstm, bidirectional = False, batch_first=True) # noqa
        self.fc = torch.nn.Linear(self.hiddendim_lstm, self.numclasses)

    def forward(self, ids, mask, token_type_ids):
      last_hidden_state, pooled_output = self.bert(ids, attention_mask = mask, token_type_ids = token_type_ids)
      #print(last_hidden_state.shape)
      #print(pooled_output.shape)
      out, _ = self.Gru(last_hidden_state, None)
      #print(out.shape)
      out = self.dropout(out[:, -1, :])
      #print(out.shape)
      out = self.fc(out)
      return out


class Bert_BiLSTM(torch.nn.Module):
    def __init__(self, numclasses):
        super(Bert_BiLSTM, self).__init__()
        self.numclasses = numclasses
        self.embeddim = embeddim
        self.numlayers = numlayers
        self.hiddendim_lstm = hiddendim_lstm
        self.dropout = torch.nn.Dropout(0.3)

        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        print("BERT Model Loaded")
        self.lstm = torch.nn.LSTM(self.embeddim, self.hiddendim_lstm, bidirectional = True, batch_first=True) # noqa
        self.fc = torch.nn.Linear(self.hiddendim_lstm*2, self.numclasses)

    def forward(self, ids, mask, token_type_ids):
      last_hidden_state, pooled_output = self.bert(ids, attention_mask = mask, token_type_ids = token_type_ids)
      #print(last_hidden_state.shape)
      #print(pooled_output.shape)
      out, _ = self.lstm(last_hidden_state, None)
      #print(out.shape)
      out = self.dropout(out[:, -1, :])
      #print(out.shape)
      out = self.fc(out)
      return out


class Bert_BiGRU(torch.nn.Module):
    def __init__(self, numclasses):
        super(Bert_BiGRU, self).__init__()
        self.numclasses = numclasses
        self.embeddim = embeddim
        self.numlayers = numlayers
        self.hiddendim_lstm = hiddendim_lstm
        self.dropout = torch.nn.Dropout(0.3)

        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        print("BERT Model Loaded")
        self.Gru = torch.nn.GRU(self.embeddim, self.hiddendim_lstm, bidirectional = True, batch_first=True) # noqa
        self.fc = torch.nn.Linear(self.hiddendim_lstm*2, self.numclasses)

    def forward(self, ids, mask, token_type_ids):
      last_hidden_state, pooled_output = self.bert(ids, attention_mask = mask, token_type_ids = token_type_ids)
      #print(last_hidden_state.shape)
      #print(pooled_output.shape)
      out, _ = self.Gru(last_hidden_state, None)
      #print(out.shape)
      out = self.dropout(out[:, -1, :])
      #print(out.shape)
      out = self.fc(out)
      return out


class Bert_Attention(torch.nn.Module):
    def __init__(self, numclasses):
        super(Bert_Attention, self).__init__()
        self.numclasses = numclasses
        self.embeddim = embeddim
        self.numlayers = numlayers
        self.hiddendim_lstm = hiddendim_lstm
        self.fchidden = fchidden
        self.dropout = torch.nn.Dropout(0.1)

        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        print("BERT Model Loaded")
        q_t = np.random.normal(loc=0.0, scale=0.1, size=(1, self.embeddim))
        self.q = nn.Parameter(torch.from_numpy(q_t)).float().to(device)
        w_ht = np.random.normal(loc=0.0, scale=0.1, size=(self.embeddim, self.fchidden)) # noqa
        self.w_h = nn.Parameter(torch.from_numpy(w_ht)).float().to(device)

        self.fc = nn.Linear(self.fchidden, self.numclasses)
    
    def forward(self, ids, mask, token_type_ids):
      last_hidden_state, pooled_output = self.bert(ids, attention_mask = mask, token_type_ids = token_type_ids)
      #print(last_hidden_state.shape)
      #print(pooled_output.shape)
      att = self.attention(last_hidden_state)
      #print("3",att.shape)
      out = self.dropout(att)
      out = self.fc(out)
      #print("la",out.shape)
      return out

    def attention(self, h):
        #print("hhhhh",h.shape)
        v = torch.matmul(self.q, h.transpose(-2, -1)).squeeze(1)
        v = torch.nn.functional.softmax(v, -1)
        v_temp = torch.matmul(v.unsqueeze(1), h).transpose(-2, -1)
        #print("tem",v_temp.shape)
        v = torch.matmul(self.w_h.transpose(1, 0), v_temp).squeeze(2)
        #print(v.shape)
        return v

class Bert_BiLAttention(torch.nn.Module):
    def __init__(self, numclasses, device):
        super(Bert_BiLAttention, self).__init__()
        self.numclasses = numclasses
        self.embeddim = embeddim
        self.numlayers = numlayers
        self.hiddendim_lstm = hiddendim_lstm
        self.fchidden = fchidden
        self.dropout = torch.nn.Dropout(0.3)

        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        print("BERT Model Loaded")
        self.lstm = torch.nn.LSTM(self.embeddim, self.hiddendim_lstm, bidirectional = True, batch_first=True)

        q_t = np.random.normal(loc=0.0, scale=0.1, size=(1, self.hiddendim_lstm*2))
        self.q = torch.nn.Parameter(torch.from_numpy(q_t)).float().to(device)
        w_ht = np.random.normal(loc=0.0, scale=0.1, size=(self.hiddendim_lstm*2, self.fchidden)) # noqa
        self.w_h = torch.nn.Parameter(torch.from_numpy(w_ht)).float().to(device)

        self.fc = torch.nn.Linear(self.fchidden, self.numclasses)

    def forward(self, inp_ids, att_mask, token_ids):
        last_hidden_state, pooler_output = self.bert(input_ids=inp_ids, attention_mask=att_mask, token_type_ids=token_ids)
        #print("1",last_hidden_state.shape)
        out, _ = self.lstm(last_hidden_state, None)
        #print("2",out.shape)
        #out = self.attention(out)
        att = self.attention(out)
        #print("3",att.shape)
        out = self.dropout(att)
        out = self.fc(out)
        #print("la",out.shape)
        return out

    def attention(self, h):
        #print("hhhhh",h.shape)
        v = torch.matmul(self.q, h.transpose(-2, -1)).squeeze(1)
        v = torch.nn.functional.softmax(v, -1)
        v_temp = torch.matmul(v.unsqueeze(1), h).transpose(-2, -1)
        #print("tem",v_temp.shape)
        v = torch.matmul(self.w_h.transpose(1, 0), v_temp).squeeze(2)
        #print(v.shape)
        return v


class Bert_BiGAttention(torch.nn.Module):
    def __init__(self, numclasses, device):
        super(Bert_BiGAttention, self).__init__()
        self.numclasses = numclasses
        self.embeddim = embeddim
        self.numlayers = numlayers
        self.hiddendim_lstm = hiddendim_lstm
        self.fchidden = fchidden
        self.dropout = torch.nn.Dropout(0.3)

        self.bert = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        print("BERT Model Loaded")
        self.Gru = torch.nn.GRU(self.embeddim, self.hiddendim_lstm, bidirectional = True, batch_first=True)

        q_t = np.random.normal(loc=0.0, scale=0.1, size=(1, self.hiddendim_lstm*2))
        self.q = torch.nn.Parameter(torch.from_numpy(q_t)).float().to(device)
        w_ht = np.random.normal(loc=0.0, scale=0.1, size=(self.hiddendim_lstm*2, self.fchidden)) # noqa
        self.w_h = torch.nn.Parameter(torch.from_numpy(w_ht)).float().to(device)

        self.fc = torch.nn.Linear(self.fchidden, self.numclasses)

    def forward(self, inp_ids, att_mask, token_ids):
        last_hidden_state, pooler_output = self.bert(input_ids=inp_ids, attention_mask=att_mask, token_type_ids=token_ids)
        #print("1",last_hidden_state.shape)
        out, _ = self.Gru(last_hidden_state, None)
        #print("2",out.shape)
        #out = self.attention(out)
        att = self.attention(out)
        #print("3",att.shape)
        out = self.dropout(att)
        out = self.fc(out)
        #print("la",out.shape)
        return out

    def attention(self, h):
        #print("hhhhh",h.shape)
        v = torch.matmul(self.q, h.transpose(-2, -1)).squeeze(1)
        v = torch.nn.functional.softmax(v, -1)
        v_temp = torch.matmul(v.unsqueeze(1), h).transpose(-2, -1)
        #print("tem",v_temp.shape)
        v = torch.matmul(self.w_h.transpose(1, 0), v_temp).squeeze(2)
        #print(v.shape)
        return v

model = Bert_Attention(numclasses)
#model = Bert_BiLAttention(numclasses, device)
#model = Bert_BiGAttention(numclasses, device)
#model = Bert_LSTM(numclasses)
#model = Bert_BiLSTM(numclasses)
#model = Bert_GRU(numclasses)
#model = Bert_BiGRU(numclasses)
#model = BERTClass()
model.to(device)
model


BERT Model Loaded


Bert_Attention(
  (dropout): Dropout(p=0.1, inplace=False)
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        #print(data)
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)
        #print(outputs.shape)
        #print(targets.shape)
        #print(outputs.dtype)
        #print(targets.dtype)
        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Loss:  0.6404091119766235
Epoch: 1, Loss:  0.4076848030090332
Epoch: 2, Loss:  0.15757513046264648
Epoch: 3, Loss:  0.12609796226024628


In [ ]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            #print(targets.shape)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
best_accuracy = 0

for epoch in range(EPOCHS):
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    precision_score_micro = metrics.precision_score(targets, outputs, average='micro')
    precision_score_macro = metrics.precision_score(targets, outputs, average='macro')
    recall_score_micro = metrics.recall_score(targets, outputs, average='micro')
    recall_score_macro = metrics.recall_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")
    print(f"precision_score (Micro) = {precision_score_micro}")
    print(f"precision_score (Macro) = {precision_score_macro}")
    print(f"recall_score (Micro) = {recall_score_micro}")
    print(f"recall_score (Macro) = {recall_score_macro}")
    if accuracy > best_accuracy:
      torch.save(model.state_dict(), 'best_model_state.bin')
      best_accuracy = accuracy

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Accuracy Score = 0.8653648509763617
F1 Score (Micro) = 0.8830227743271222
F1 Score (Macro) = 0.8177090736573126
precision_score (Micro) = 0.8894681960375391
precision_score (Macro) = 0.811232663005857
recall_score (Micro) = 0.8766700924974307
recall_score (Macro) = 0.827077879394988
Accuracy Score = 0.8653648509763617
F1 Score (Micro) = 0.8830227743271222
F1 Score (Macro) = 0.8177090736573126
precision_score (Micro) = 0.8894681960375391
precision_score (Macro) = 0.811232663005857
recall_score (Micro) = 0.8766700924974307
recall_score (Macro) = 0.827077879394988
Accuracy Score = 0.8653648509763617
F1 Score (Micro) = 0.8830227743271222
F1 Score (Macro) = 0.8177090736573126
precision_score (Micro) = 0.8894681960375391
precision_score (Macro) = 0.811232663005857
recall_score (Micro) = 0.8766700924974307
recall_score (Macro) = 0.827077879394988
Accuracy Score = 0.8653648509763617
F1 Score (Micro) = 0.8830227743271222
F1 Score (Macro) = 0.8177090736573126
precision_score (Micro) = 0.88946819

In [ ]:
class_names = ['Negative', 'Neutral', 'Positive']
print(classification_report(outputs, targets, target_names=class_names))


              precision    recall  f1-score   support

    Negative       0.91      0.80      0.85       251
     Neutral       0.68      0.67      0.68        95
    Positive       0.89      0.96      0.93       613

   micro avg       0.88      0.89      0.88       959
   macro avg       0.83      0.81      0.82       959
weighted avg       0.88      0.89      0.88       959
 samples avg       0.88      0.87      0.87       959



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#class_names = ['Class_0', 'Class_1', 'Class_2']

class_names = ['Negative', 'Neutral', 'Positive']

#test_string = "The shells were crisp and authentic, the filling was real ricotta cheese, not the fake junk a lot of Italian Places use these days."
#test_string = "the only thing more wonderful than the food ( which is exceptional ) is the service"
#test_string = "The pizza and garlic bread are good, but the staff members are horrible to us."
#test_string = "The pizza was not so cheap."
#test_string = "The pizza and pasta are delicious, but the taste of garlic bread is not good."
test_string = "The pizza is good."
#at = "staff members"
at = "food"
#at = "price"
#at = "service"


inputs = tokenizer.encode_plus(
            test_string,
            at,
            add_special_tokens=True,
            max_length=80,
            pad_to_max_length=True,
            return_token_type_ids=True,
            return_tensors='pt',
)

input_ids = inputs['input_ids'].to(device)
mask = inputs['attention_mask'].to(device)
token_type_ids = inputs["token_type_ids"].to(device)
print(input_ids)
print(mask)
print(token_type_ids)

target,  = model(input_ids, mask, token_type_ids)
#output = model(input_ids, attention_mask)

predict = torch.sigmoid(target)
print(predict)

predict = predict.cpu().detach().flatten().numpy()
predictlabel = []

for i, labelname in enumerate(class_names):
  labelprob = predict[i]
  if labelprob > 0.5:
    predictlabel.append(labelname)
predictlabel

print(predictlabel)

tensor([[  101,  1996, 10733,  2003,  2204,  1012,   102,  2833,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]],
       device='cuda:0')
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')
tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
